In [34]:
import os
import json
import openai
import gradio as gr
from dotenv import load_dotenv


In [35]:
ticket_prices = {'manila': 'NZ$1000','paris': 'NZ$2000', 'tokyo': 'NZ$2500'} 

In [36]:
def get_prices(destination_city):
    print(f'get price:{destination_city}')
    ticket_price = ticket_prices.get(destination_city.lower(),'Unknown')
    return ticket_price

In [37]:
def fxn_tool_call(message):        
        tool_call = message.tool_calls[0]
        print(f'tool_calls[0]:{tool_call}')
        arguments = json.loads(tool_call.function.arguments)
        
        print(f'fxn_tool_call arguments:{arguments}')
        city = arguments.get('destination_city')
        price = get_prices(city)
        response = {
            'role': 'tool',
            'content': json.dumps({'destination_city': city, 'price': price}),
            'tool_call_id': message.tool_calls[0].id
        }
        return response, city

In [38]:
class TicketingApp:
    def __init__(self):
        load_dotenv()
        openai.key = os.getenv('OPENAI_API_KEY')

        self.system_prompt = """
            You are a helpful assistant for a ticketing company called TicketingAI.
            Give a short and friendly answer with no more than 1 sentence.
            Always be accurate and reply 'I dont know the answer', if you dont know the answer.
        """
        self.price_function = {
            'name': 'get_prices',
            'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
            'parameters':{
                'type': 'object',
                'properties': {
                    'destination_city': {
                        'type': 'string',
                        'description': 'The destination city for the traveller'
                    }
                },
                'required': ['destination_city'],
                'additionalProperties': False

            }
        }

        self.tools = [{'type': 'function', 'function': self.price_function}]
    

    def chat(self,message, history):
        messages = [{'role': 'system','content': self.system_prompt}]
        for user, assistant in history:
            messages.append({'role': 'user','content': user})
            messages.append({'role':'assistant', 'content': assistant})
        messages.append({'role': 'user','content': message})

        completions=openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=messages,
            tools=self.tools
        )

        if completions.choices[0].finish_reason=='tool_calls':
            message = completions.choices[0].message
            response, city = fxn_tool_call(message)
            messages.append(message)
            messages.append(response)
            completions=openai.chat.completions.create(
                    model='gpt-4o-mini',
                    messages=messages
             )

        self.result = completions.choices[0].message.content
        yield self.result
        return self

    def UI(self):
        gr.ChatInterface(self.chat).launch()
        return self

In [39]:
TicketingApp().UI()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


tool_calls[0]:ChatCompletionMessageToolCall(id='call_kFFFFOYb2a3n0XG2HEYS83Ag', function=Function(arguments='{"destination_city":"Tokyo"}', name='get_prices'), type='function')
fxn_tool_call arguments:{'destination_city': 'Tokyo'}
get price:Tokyo
